In [6]:
%load_ext autoreload
%autoreload 2
import gc
import numpy as np
import pandas as pd
import sys
import re
from glob import glob
import os
HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
#  sys.path.append(f"/mnt/c/Git/go/kaggle/github/library/")
import utils
from utils import logger_func
try:
    if not logger:
        logger=logger_func()
except NameError:
    logger=logger_func()
    
from tqdm import tqdm
import joblib

#========================================================================
# Global Variable
sys.path.append(f"../py")
from info_home_credit import hcdr_key_cols
key, target, ignore_list = hcdr_key_cols()
#========================================================================

#========================================================================
# BigQuery
from google.cloud import bigquery
bq_client = bigquery.Client.from_service_account_json(f'{HOME}/privacy/horikoshi-ml-a13ef6f2f937.json')
#========================================================================

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
# Delete Datset
project = 'horikoshi-ml-224313'
dataset = 'hori'
table = 'tmp'
bq_client.delete_table(f'{dataset}.{table}')

In [19]:
#========================================================================
# Create New Table from Query
query = """
select
*
,dense_rank() over (partition by SK_ID_CURR order by DAYS_INSTALMENT ) as asc_rank_curr
,dense_rank() over (partition by SK_ID_PREV order by DAYS_INSTALMENT ) as asc_rank_prev
,dense_rank() over (partition by SK_ID_CURR order by DAYS_INSTALMENT DESC) as desc_rank_curr
,dense_rank() over (partition by SK_ID_PREV order by DAYS_INSTALMENT DESC) as desc_rank_prev
from hori.installment
;
"""

project = 'horikoshi-ml-224313'
dataset = 'hori'
new_table = 'ins_rank'
bq_cmd = f"bq --location=US query --destination_table {project}:{dataset}.{new_table} --use_legacy_sql=false '{query}'"
os.system(bq_cmd)
#========================================================================

0

In [27]:
from google.cloud import bigquery

def bq_load(query):
    bq_client = bigquery.Client.from_service_account_json(f'{HOME}/privacy/horikoshi-ml-a13ef6f2f937.json')
    df = bq_client.query(query).to_dataframe()
    return df


df = bq_load(query)
print(df.shape)

# prefix = 'f008_big_ins-cur-l3-'
# prefix = 'f008_big_ins-pre-f3-'
prefix = 'f008_big_ins-pre-l3-'
base = utils.read_df_pkl('../input/base0*').set_index(key)
df.set_index(key, inplace=True)
df = base.join(df)
utils.save_feature(df_feat=df, ignore_list=ignore_list, is_train=2, prefix=prefix, target=target)

100%|██████████| 3/3 [00:00<00:00, 602.98it/s]

(339587, 36)


(307511,) | NUM_INSTALMENT_VERSION_sum
(307511,) | NUM_INSTALMENT_NUMBER_sum
(307511,) | DAYS_ENTRY_PAYMENT_sum
(307511,) | DAYS_INSTALMENT_sum
(307511,) | AMT_INSTALMENT_sum
(307511,) | AMT_PAYMENT_sum
(307511,) | DIFF_PAYMENT_sum
(307511,) | NUM_INSTALMENT_VERSION_mean
(307511,) | NUM_INSTALMENT_NUMBER_mean
(307511,) | DAYS_ENTRY_PAYMENT_mean
(307511,) | DAYS_INSTALMENT_mean
(307511,) | AMT_INSTALMENT_mean
(307511,) | AMT_PAYMENT_mean
(307511,) | DIFF_PAYMENT_mean
(307511,) | NUM_INSTALMENT_VERSION_max
(307511,) | NUM_INSTALMENT_NUMBER_max
(307511,) | DAYS_ENTRY_PAYMENT_max
(307511,) | DAYS_INSTALMENT_max
(307511,) | AMT_INSTALMENT_max
(307511,) | AMT_PAYMENT_max
(307511,) | DIFF_PAYMENT_max
(307511,) | NUM_INSTALMENT_VERSION_min
(307511,) | NUM_INSTALMENT_NUMBER_min
(307511,) | DAYS_ENTRY_PAYMENT_min
(307511,) | DAYS_INSTALMENT_min
(307511,) | AMT_INSTALMENT_min
(307511,) | AMT_PAYMENT_min
(307511,) | DIFF_PAYMENT_min
(307511,) | NUM_INSTALMENT_VERSION_std
(307511,) | NUM_INSTALMENT

In [ ]:
exec_date = '20190301'
queries = {}
query_file_list = glob.glob("./sql/{exec_date}*.sql")
for query_file in query_list:
    with open(query_file) as f:
        query = f.read()
    feature_name = utils.get_filename(query, delimiter='sql').replace(f'{exec_date}_', '')
    queries[feature_name] = query

r = joblib.Parallel(n_jobs=32)(
    joblib.delayed(bq_load)(query, feature_name, exec_date)
    for feature_name, query in queries.items()
)